# Example: Create GSEA figures for selected genesets

In [1]:
from src.epimedcore.analysis.gsea import GseaPrerank
from src.epimedcore.entity import Project
from src.epimedcore.plot.gsea import GseaPlot
from src.epimedcore.service import FigureService
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def cut_text(text, max_length=25, sep='_'):
    text_length = len(text) 
    if text_length>max_length:
        positions = [pos for pos, char in enumerate(text) if char == sep]
        best_pos = positions[0]
        min_distance = text_length
        middle_pos = text_length/2.5
        for pos in positions:
            distance = abs(pos-middle_pos)            
            if (distance<min_distance):
                min_distance=distance
                best_pos = pos
        text = text[:best_pos] + '\n' + text[best_pos+1:]
    return text

## Create a project

In [3]:
project = Project(name='GSEA', root_dir='./example')
project.dump()
print(project)

Project [name=GSEA, root_dir=./example/, project_dir=./example/GSEA/, data_dir=./example/GSEA/DATA/, results_dir=./example/GSEA/RESULTS/, figures_dir=./example/GSEA/FIGURES/, json_file=./example/GSEA/project.json, datasets={}]


## Import all MSigDB genesets

In [4]:
msigdb_file = 'msigdb.v7.5.1.json'
gsea_prerank = GseaPrerank(project=project)
gsea_prerank.import_genesets_from_json(json_file=project.data_dir + msigdb_file)    
print(gsea_prerank)

GseaPrerank [name = GSEA Prerank, nb_genesets = 32880, rnk = (0, 0)]


## Select usefull genesets

In [5]:
geneset_names = ['HALLMARK_HYPOXIA', 'QI_HYPOXIA']
genesets = {k: gsea_prerank.genesets[k] for k in geneset_names}
print(genesets)

{'HALLMARK_HYPOXIA': ['ACKR3', 'ADM', 'ADORA2B', 'AK4', 'AKAP12', 'ALDOA', 'ALDOB', 'ALDOC', 'AMPD3', 'ANGPTL4', 'ANKZF1', 'ANXA2', 'ATF3', 'ATP7A', 'B3GALT6', 'B4GALNT2', 'BCAN', 'BCL2', 'BGN', 'BHLHE40', 'BNIP3L', 'BRS3', 'BTG1', 'CA12', 'CASP6', 'CAV1', 'CAVIN1', 'CAVIN3', 'CCN1', 'CCN2', 'CCN5', 'CCNG2', 'CDKN1A', 'CDKN1B', 'CDKN1C', 'CHST2', 'CHST3', 'CITED2', 'COL5A1', 'CP', 'CSRP2', 'CXCR4', 'DCN', 'DDIT3', 'DDIT4', 'DPYSL4', 'DTNA', 'DUSP1', 'EDN2', 'EFNA1', 'EFNA3', 'EGFR', 'ENO1', 'ENO2', 'ENO3', 'ERO1A', 'ERRFI1', 'ETS1', 'EXT1', 'F3', 'FAM162A', 'FBP1', 'FOS', 'FOSL2', 'FOXO3', 'GAA', 'GALK1', 'GAPDH', 'GAPDHS', 'GBE1', 'GCK', 'GCNT2', 'GLRX', 'GPC1', 'GPC3', 'GPC4', 'GPI', 'GRHPR', 'GYS1', 'HAS1', 'HDLBP', 'HEXA', 'HK1', 'HK2', 'HMOX1', 'HOXB9', 'HS3ST1', 'HSPA5', 'IDS', 'IER3', 'IGFBP1', 'IGFBP3', 'IL6', 'ILVBL', 'INHA', 'IRS2', 'ISG20', 'JMJD6', 'JUN', 'KDELR3', 'KDM3A', 'KIF5A', 'KLF6', 'KLF7', 'KLHL24', 'LALBA', 'LARGE1', 'LDHA', 'LDHC', 'LOX', 'LXN', 'MAFF', 'MAP3K1',

## Import RNK file

In [6]:
rnk = pd.read_csv(project.data_dir + 'GSE106291_AML_diff_mean.rnk', sep='\t')
rnk.head()

,gene,rank
0,APP,2.589942
1,LTF,2.296218
2,PRTN3,1.838293
3,H1-0,1.820125
4,BEX2,1.805023


## Perform GSEA Prerank analysis

In [7]:
prerank_options = {
    'min_size': 0, 
    'max_size': 1000, 
    'permutation_num': 1000, 
    'weighted_score_type': 1,
    'processes': -1,  
    'no_plot': True,
    'seed': 0,
    'verbose': True
    }

In [8]:
gsea_options = {
        'project': project,
        'rnk': rnk, 
        'genesets': genesets, 
        'prerank_options': prerank_options
        }

In [9]:
gsea_prerank = GseaPrerank(**gsea_options)
print(gsea_prerank)

GseaPrerank [name = GSEA Prerank, nb_genesets = 2, rnk = (21045, 2)]


In [10]:
gsea_prerank.perform()

2022-07-01 19:50:12,818 Parsing data files for GSEA.............................
2022-07-01 19:50:12,914 0000 gene_sets have been filtered out when max_size=1000 and min_size=0
2022-07-01 19:50:12,915 0002 gene_sets used for further statistical testing.....
2022-07-01 19:50:12,915 Start to run GSEA...Might take a while..................
2022-07-01 19:50:14,416 Start to generate gseapy reports, and produce figures...
2022-07-01 19:50:14,421 Congratulations. GSEApy runs successfully................



## Generate GSEA figures

In [11]:
for geneset_name in geneset_names:
    fig = plt.figure(figsize=(5, 4))
    plot_options = {
        'gsea_prerank': gsea_prerank,
        'geneset_name': geneset_name,
        'title': 'GSE106291' + '\n' + cut_text(geneset_name),
        'pos_label': 'GEC 3-5',
        'neg_label': 'GEC 0',
        # 'pvalue': pvalue,
        # 'fdr': fdr,
        # 'nes': nes, 
        }
    gsea_plot = GseaPlot(fig=fig, **plot_options)
    gsea_plot.plot()
    plt.close(fig)
    file_prefix = geneset_name
    FigureService.save_fig_with_resolution(fig, project.figures_dir, file_prefix, dpi=300, ext='pdf')
    FigureService.save_fig_with_resolution(fig, project.figures_dir, file_prefix, dpi=300, ext='png')
print('Figures generated in', project.figures_dir)

Figures generated in ./example/GSEA/FIGURES/
